In [1]:
import warnings
warnings.filterwarnings("ignore")  # Optional: to suppress warnings

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt


In [2]:
def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    
    log_model = LogisticRegression()
    RF = RandomForestClassifier()
    DT = DecisionTreeClassifier()
    svc_model = SVC(kernel='linear')  # ✅ FIXED: Linear SVM for RFE

    rfemodellist = [log_model, svc_model, RF, DT] 
    for model in rfemodellist:
        print(model)
        rfe = RFE(model, n)
        rfe_fit = rfe.fit(indep_X, dep_Y)
        selected_features = rfe_fit.transform(indep_X)
        rfelist.append(selected_features)

    return rfelist


In [3]:
def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test


In [4]:
def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, X_test, y_test, cm


In [5]:
def logistic_gridsearch(X_train, y_train, X_test, y_test):       
    param_grid = {
        'penalty': ['l2'],
        'class_weight': ['balanced'],
        'multi_class': ['ovr', 'auto'],
        'solver': ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']
    }
    grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    return cm_prediction(best_model, X_test, y_test)


In [6]:
def svm_linear_gridsearch(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}
    grid = GridSearchCV(SVC(kernel='linear'), param_grid, cv=5)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    return cm_prediction(best_model, X_test, y_test)


In [7]:
def svm_NL_gridsearch(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}
    grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    return cm_prediction(best_model, X_test, y_test)


In [8]:
def Navie(X_train, y_train, X_test, y_test):
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)


In [9]:
def knn_gridsearch(X_train, y_train, X_test, y_test):
    param_grid = {
        'n_neighbors': [5],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'metric': ['minkowski']
    }
    grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    return cm_prediction(best_model, X_test, y_test)


In [10]:
def Decision_gridsearch(X_train, y_train, X_test, y_test):
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_features': ['sqrt', 'log2'],
        'splitter': ['best', 'random']
    }
    grid = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    return cm_prediction(best_model, X_test, y_test)


In [11]:
def random_gridsearch(X_train, y_train, X_test, y_test):
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_features': ['sqrt', 'log2'],
        'n_estimators': [10, 100]
    }
    grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    return cm_prediction(best_model, X_test, y_test)


In [12]:
def rfe_classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf): 
    rfedataframe = pd.DataFrame(index=['Logistic', 'SVC', 'Random', 'DecisionTree'],
                                columns=['Logistic', 'SVMl', 'SVMnl', 'KNN', 'Navie', 'Decision', 'Random'])

    for i, idx in enumerate(rfedataframe.index):
        rfedataframe.loc[idx] = [
            acclog[i], accsvml[i], accsvmnl[i], accknn[i],
            accnav[i], accdes[i], accrf[i]
        ]
    return rfedataframe


In [13]:
dataset1 = pd.read_csv("Preprocessed_Loan_Default.csv")
df2 = pd.get_dummies(dataset1.copy(), drop_first=True)

indep_X = df2.drop('Loan_Status_Non-Default', axis=1)
dep_Y = df2['Loan_Status_Non-Default']

rfelist = rfeFeature(indep_X, dep_Y, 3)  # Get features selected by different models

# Store accuracy for each RFE model
acclog = []
accsvml = []
accsvmnl = []
accknn = []
accnav = []
accdes = []
accrf = []

for features in rfelist:
    X_train, X_test, y_train, y_test = split_scalar(features, dep_Y)
    
    _, acc1, _, _, _, _ = logistic_gridsearch(X_train, y_train, X_test, y_test)
    _, acc2, _, _, _, _ = svm_linear_gridsearch(X_train, y_train, X_test, y_test)
    _, acc3, _, _, _, _ = svm_NL_gridsearch(X_train, y_train, X_test, y_test)
    _, acc4, _, _, _, _ = knn_gridsearch(X_train, y_train, X_test, y_test)
    _, acc5, _, _, _, _ = Navie(X_train, y_train, X_test, y_test)
    _, acc6, _, _, _, _ = Decision_gridsearch(X_train, y_train, X_test, y_test)
    _, acc7, _, _, _, _ = random_gridsearch(X_train, y_train, X_test, y_test)

    acclog.append(acc1)
    accsvml.append(acc2)
    accsvmnl.append(acc3)
    accknn.append(acc4)
    accnav.append(acc5)
    accdes.append(acc6)
    accrf.append(acc7)

result_df = rfe_classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
print(result_df)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators='warn',
                       n_jobs=None, oob_score=False, random_

C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version inst

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version inst

C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: Depre

...

C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\neighbors\base.py:441: Depre

C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version inst

C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version inst

             Logistic    SVMl   SVMnl     KNN   Navie Decision  Random
Logistic       0.5572  0.7996  0.7996  0.7996  0.7996   0.7996  0.7996
SVC            0.5788  0.7996  0.7996   0.772  0.7996   0.6756   0.676
Random           0.49  0.7996  0.7996  0.7628  0.7996   0.6872   0.792
DecisionTree   0.4972  0.7996  0.7996  0.7608  0.7996   0.6712  0.7856


C:\Anaconda\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [14]:
result_df

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Logistic,0.5572,0.7996,0.7996,0.7996,0.7996,0.7996,0.7996
SVC,0.5788,0.7996,0.7996,0.772,0.7996,0.6756,0.676
Random,0.49,0.7996,0.7996,0.7628,0.7996,0.6872,0.792
DecisionTree,0.4972,0.7996,0.7996,0.7608,0.7996,0.6712,0.7856
